In [309]:
import pandas as pd

data = pd.read_excel('data_set.xlsx')

In [310]:
def replace_binary_value(b):
    if b and pd.notnull(b):
        return 1
    else:
        return -1

data.iloc[:, 12:16] = data.iloc[:, 12:16].agg(lambda row: row.agg(replace_binary_value, axis=0), axis=1)

data_kal_obl = data[data['REGION'] == 'Калужская область']
data_tul_obl = data[data['REGION'] == 'Тульская область']

In [311]:
import seaborn as sns

#sns.pairplot(data_kal_obl.iloc[:, 3:17])

In [312]:
data_kaluga = data_kal_obl[data_kal_obl['CITY'] == 'КАЛУГА']
data_tula = data_tul_obl[data_tul_obl['CITY'] == 'ТУЛА']
data_kal_obl = data_kal_obl[data_kal_obl['CITY'] != 'КАЛУГА']
data_tul_obl = data_tul_obl[data_tul_obl['CITY'] != 'ТУЛА']

In [313]:
X = data_kaluga[data_kaluga['cashbox_daily'].notnull()].iloc[:, 6:18]

In [314]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 57 entries, 0 to 239
Data columns (total 11 columns):
cashbox_MAX                57 non-null float64
cashbox_trend              55 non-null float64
hh_500                     57 non-null float64
value1                     57 non-null int64
value2                     57 non-null int64
POPULATION                 57 non-null int64
NEAR_Malls                 57 non-null int64
NEAR_Business_centers      57 non-null int64
NEAR_metro_rjd             57 non-null int64
NEAR_Stations              57 non-null int64
macro_salary_avg_yearly    57 non-null int64
dtypes: float64(3), int64(8)
memory usage: 5.3 KB


In [315]:
del X['POPULATION']
del X['macro_salary_avg_yearly']

In [316]:
import numpy as np

X.fillna(0, inplace=True)
X = (X - X.mean()) / X.std()

In [317]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse

lasso = Lasso(0.05)
ridge = Ridge()
lin_reg = LinearRegression()

In [318]:
y = data_kaluga[data_kaluga['cashbox_daily'].notnull()].cashbox_daily
y = (y - y.mean()) / y.std()

In [319]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

lasso.fit(X_train, y_train)
print(mse(y_test, lasso.predict(X_test)), mae(y_test, lasso.predict(X_test)))

0.16024551166537843 0.3252978567701648


In [320]:
lasso.fit(X, y)
lasso.coef_

array([ 0.7950587 ,  0.        ,  0.        ,  0.04516096,  0.05593081,
       -0.        ,  0.        ,  0.01609916, -0.06898949])

In [321]:
lasso.fit(X.drop(labels='value1', axis=1), y)
lasso.coef_

array([ 0.7949395 ,  0.        ,  0.        ,  0.09664327, -0.        ,
        0.        ,  0.01323896, -0.07149039])

In [322]:
lasso.fit(X.drop(labels='value2', axis=1), y)
lasso.coef_

array([ 0.79889328,  0.        ,  0.00198145,  0.09487306, -0.        ,
        0.        ,  0.01836106, -0.0664145 ])

In [323]:
X.drop(labels=['cashbox_trend', 'value2', 'NEAR_Malls', 'NEAR_Business_centers'], axis=1, inplace=True)

In [324]:
X.head()

,cashbox_MAX,hh_500,value1,NEAR_metro_rjd,NEAR_Stations
0,1.494381,1.675132,1.290828,-0.132453,1.044805
2,1.212329,0.744297,1.307995,-0.132453,-0.940325
3,1.212329,1.443183,1.290828,-0.132453,-0.940325
4,0.366173,1.055299,1.239327,-0.132453,1.044805
6,-0.479983,1.678173,1.101992,-0.132453,1.044805


In [325]:
def test_regressor(X, y, regressor):
    mean_mse = 0
    mean_mae = 0
    
    for i in range(100):
        X_train , X_test, y_train, y_test = train_test_split(X, y)
        regressor.fit(X_train, y_train)
        mean_mse += mse(y_test, regressor.predict(X_test))
        mean_mae += mae(y_test, regressor.predict(X_test))
        
    print('MSE = ', mean_mse/100)
    print('MAE = ', mean_mae/100)

In [326]:
print('LinReg:')
test_regressor(X, y, lin_reg)

print('Lasso:')
test_regressor(X, y, lasso)

print('Ridge:')
test_regressor(X, y, ridge)

LinReg:
MSE =  0.24347470746358676
MAE =  0.38496679786276344
Lasso:
MSE =  0.23463223531453262
MAE =  0.38075487736347896
Ridge:
MSE =  0.2313854768985617
MAE =  0.37670062304120777


In [327]:
from sklearn.tree import DecisionTreeRegressor

des_tree = DecisionTreeRegressor()

print('DesTree:')
test_regressor(X, y, des_tree)

DesTree:
MSE =  0.4265439014459107
MAE =  0.4918268477058195


In [328]:
from sklearn.tree import ExtraTreeRegressor

extra_tree = ExtraTreeRegressor()

print('ExtraTree:')
test_regressor(X, y, extra_tree)

ExtraTree:
MSE =  0.5051163225879114
MAE =  0.5286707460835273


In [329]:
from sklearn.neighbors import KNeighborsRegressor

knn = KNeighborsRegressor()

print('KNN::')
test_regressor(X, y, knn)

KNN::
MSE =  0.34381647038439234
MAE =  0.4462501614998645


In [330]:
def getPredict(data, regressor):
    X = data[data['cashbox_daily'].notnull()].iloc[:, 6:18]
    X.drop(labels=['cashbox_trend', 'value2', 'NEAR_Malls',\
                   'NEAR_Business_centers', 'POPULATION', 'macro_salary_avg_yearly'], axis=1, inplace=True)
    y = data[data['cashbox_daily'].notnull()].cashbox_daily

    city_regressor = regressor()
    city_regressor.fit(X, y)
    
    df = data[data['cashbox_daily'].isnull()].iloc[:, 6:18]
    df['cashbox_MAX'] = 0
    df.drop(labels=['cashbox_trend', 'value2', 'NEAR_Malls',\
                    'NEAR_Business_centers', 'POPULATION', 'macro_salary_avg_yearly'], axis=1, inplace=True)

    return city_regressor.predict(df)

In [334]:
getPredict(data_kaluga, Ridge)

array([ 0.04060296,  0.19420493,  0.05736566,  0.16822705,  0.44070163,
        0.01207763,  0.06830274, -0.25262999, -0.30555644])